In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None
import requests
import re
from bs4 import BeautifulSoup
import json

In [2]:
def findQuantity(text):
    unit="(g|kg|l|ml| litro)"
    x = re.search("[0-9,]+"+unit, text, re.IGNORECASE)

    return x.group()

In [3]:
def findRelPrice(price, quantity):
    qt = int(''.join(filter(str.isdigit, quantity)))
    unit = ''.join(filter(str.isalpha, quantity))

    relPrice = float(price)/qt

    if unit == 'g' or unit == 'ml':
        relPrice*=1000
    
    return str(round(relPrice, 2)) +  ' R$/' + ("kg" if unit == 'g' else 'L')

In [150]:
baseURL = "https://www.sondadelivery.com.br/delivery/busca/"

wh = 'WH-EFB'

products = [
    "Atum solido Gomes da Costa natural",
    "Torrada Bauducco",
    "Suco Do Bem Goiaba",
    "Milho verde em conserva",
]

In [181]:
results = pd.DataFrame(columns=['searchKey', 'productName', 'price', 'quantity', 'relPrice'])

for key in products:
    url = baseURL + key.lower().replace(' ', '%20')
    html = requests.get(url).text
    soup = BeautifulSoup(html, "html.parser")
    items = soup.find_all("li", class_="nm-product-item is-market-product nm-with-quantity")
    for i, item in enumerate(items):
        info = json.loads(item["data-product-attr"])
        qt = findQuantity(info['productName'])
        price = float(info['price'].replace(',','.'))
        if wh in info['warehouses']:
            results.loc[key.lower().replace(' ', '_') + '_' + str(i)] = [key, info['productName'], price, qt, findRelPrice(price, qt)]
        else:
            print(info['productName'] + " não disponível")
            
results

Torrada Integral Bauducco 142g não disponível
Torrada Bauducco Light 142g não disponível
Torrada Multigrãos Bauducco 142g não disponível
Torrada Multigrãos Bauducco 284g com 2 Unidades não disponível
Torrada Tradicional Bauducco 142g não disponível
Torrada Bauducco Tradicional 284g não disponível
Milho Verde em Conserva Light Bonduelle 225g não disponível


,searchKey,productName,price,quantity,relPrice
atum_solido_gomes_da_costa_natural_0,Atum solido Gomes da Costa natural,Atum Sólido ao Natural Gomes da Costa 170g,8.69,170g,51.12 R$/kg
atum_solido_gomes_da_costa_natural_1,Atum solido Gomes da Costa natural,Atum Sólido ao Natural Gomes da Costa 170g,8.59,170g,50.53 R$/kg
atum_solido_gomes_da_costa_natural_2,Atum solido Gomes da Costa natural,Atum Sólido ao Natural Light Gomes da Costa Pr...,9.05,170g,53.24 R$/kg
torrada_bauducco_2,Torrada Bauducco,Torrada Salgada Bauducco 110g,3.69,110g,33.55 R$/kg
suco_do_bem_goiaba_0,Suco Do Bem Goiaba,Suco de Goiaba Do Bem Todo Dia 1 Litro,7.39,1 Litro,7.39 R$/L
suco_do_bem_goiaba_1,Suco Do Bem Goiaba,Suco de Goiaba Do Bem Todo Dia 200ml,1.99,200ml,9.95 R$/L
milho_verde_em_conserva_0,Milho verde em conserva,Milho Verde em Conserva Knorr 290g,2.09,290g,7.21 R$/kg
milho_verde_em_conserva_1,Milho verde em conserva,Milho Verde em Conserva Carrefour 200g,2.09,200g,10.45 R$/kg
milho_verde_em_conserva_2,Milho verde em conserva,Milho Verde em Conserva Bonduelle 170g,2.49,170g,14.65 R$/kg
milho_verde_em_conserva_3,Milho verde em conserva,Milho Verde em Conserva Bonduelle ao Natural 200g,3.39,200g,16.95 R$/kg


In [182]:
suggested = results.drop_duplicates(subset=['searchKey'], keep='first')
suggested

,searchKey,productName,price,quantity,relPrice
atum_solido_gomes_da_costa_natural_0,Atum solido Gomes da Costa natural,Atum Sólido ao Natural Gomes da Costa 170g,8.69,170g,51.12 R$/kg
torrada_bauducco_2,Torrada Bauducco,Torrada Salgada Bauducco 110g,3.69,110g,33.55 R$/kg
suco_do_bem_goiaba_0,Suco Do Bem Goiaba,Suco de Goiaba Do Bem Todo Dia 1 Litro,7.39,1 Litro,7.39 R$/L
milho_verde_em_conserva_0,Milho verde em conserva,Milho Verde em Conserva Knorr 290g,2.09,290g,7.21 R$/kg


In [183]:
min_price = results.groupby('searchKey').min()
min_price

,productName,price,quantity,relPrice
searchKey,,,,
Atum solido Gomes da Costa natural,Atum Sólido ao Natural Gomes da Costa 170g,8.59,170g,50.53 R$/kg
Milho verde em conserva,Milho Verde em Conserva Bonduelle 170g,2.09,170g,10.45 R$/kg
Suco Do Bem Goiaba,Suco de Goiaba Do Bem Todo Dia 1 Litro,1.99,1 Litro,7.39 R$/L
Torrada Bauducco,Torrada Salgada Bauducco 110g,3.69,110g,33.55 R$/kg


In [193]:
def calculatePrices(chosen):
    chosen['amount'] = 0
    print("Quantos de cada um dos itens abaixo você deseja comprar?")
    for i,item in chosen.iterrows(): 
        print(item['productName'] + " (R$ " + str(item['price']) + ")")
        chosen.loc[i,'amount'] = int(input())

    chosen['totalPrice'] = chosen.amount*chosen.price
    return chosen

In [195]:
calculatePrices(min_price)

Quantos de cada um dos itens abaixo você deseja comprar?
Atum Sólido ao Natural Gomes da Costa 170g (R$ 8.59)


KeyboardInterrupt: Interrupted by user

</br></br></br></br></br></br></br></br>

In [14]:
url = "https://www.extra.com.br/rb_f3050260-dc4b-4886-92bb-2b9e161d9a53?type=js&session=29%2415FABB1026D1C4719650DA8E293FDA26%7C_default%7C1%7Cda1a5fc781fd282f%7C0&svrid=29&flavor=post&visitID=UFFWFPCFKOIBORKTQODIHKCFWLVOHMPQ-0&modifiedSince=1590454895444&referer=https%3A%2F%2Fwww.extra.com.br%2Fbauducco%2Fb&app=da1a5fc781fd282f&end=1"


In [15]:
html = requests.get(url)

ConnectionError: ('Connection aborted.', TimeoutError(10060, 'Uma tentativa de conexão falhou porque o componente conectado não respondeu\r\ncorretamente após um período de tempo ou a conexão estabelecida falhou\r\nporque o host conectado não respondeu', None, 10060, None))

In [ ]:
html.text